In [1]:
from nanotag.data import NanotagData, ImageFileCollection, Summary
from nanotag.image import ImageSeries, GaussianFilterSlider
from nanotag.tags import PointTags, PointTagSeries
from nanotag.timeline import Timeline, TimelineTags, EditTimelineTags
from nanotag.canvas import Canvas, ToolBox
from nanotag.tools import EditPointTags, ResetView, BoxZoomTool, PanZoomTool
from nanotag.utils import link
from nanotag.histogram import Histogram
from traitlets import directional_link

from bqplot import ColorScale, Scatter, Figure
import ipywidgets as widgets
import os
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
root_directory = 'D:/data/hBN_export'
analysis_file = 'ml_analysis.json'
filt = '**/*.tif'

In [3]:
#import json
#with open('D:/data/hBN_export/ml_analysis.json', 'r') as f:
#    json.load(f)

In [3]:
image_file_collection = ImageFileCollection(filter=filt)

image_file_collection.paths

['']

In [53]:
atom_tags = PointTagSeries(data_fields=('sublattice','defect'), enable_move=False)
defect_tags = PointTagSeries(data_fields=('type',), enable_move=False)

has_defect_tags = TimelineTags(row='defect', data_fields=('defect',), enable_move=False, color_scale=ColorScale(min=0, max=1))
area_tags = TimelineTags(row='scan err', data_fields=('area',), enable_move=False, color_scale=ColorScale(colors=['red', 'white'], min=.8, max=1))
skipped_tags = TimelineTags(row='skipped', data_fields=('skipped',), enable_move=False, color_scale=ColorScale(min=.8, max=1))

image_series = ImageSeries()
gaussian_filter = GaussianFilterSlider(max=10)
image_series.filters = [gaussian_filter]
nanotag_data = NanotagData(root_directory = root_directory,
                           analysis_folder = 'new_analysis',
                           read_file = analysis_file,
                           #write_file = analysis_file,
                           tags = {'points' : atom_tags, 'defects' : defect_tags, 'area': area_tags}
                           )

image_file_collection = ImageFileCollection(filter=filt, image_series=image_series)
link((nanotag_data, 'root_directory'), (image_file_collection, 'root_directory'))

def transform_file_name(x):
    return os.path.splitext(os.path.split(x)[-1])[0] + '.json'

directional_link((image_file_collection, 'path'), (nanotag_data, 'read_file'), transform=transform_file_name)
#link((image_file_collection, 'hash'), (nanotag_data, 'identifier'), check_broken=False)

timeline = Timeline()
timeline.tags = [has_defect_tags, area_tags, skipped_tags]
link((image_series, 'num_frames'), (timeline, 'num_frames'))
directional_link((timeline, 'frame_index'), (image_series, 'frame_index'))
directional_link((timeline, 'frame_index'), (atom_tags, 'frame_index'))



canvas = Canvas()
canvas.tags = [atom_tags, defect_tags]
canvas.image = image_series

toolbox = ToolBox(canvas)
toolbox.tools = {'Reset' : ResetView(),
                 'BoxZoom' : BoxZoomTool(),
                 'Pan' : PanZoomTool(),
                 'B defect' : EditPointTags(defect_tags, data_fields={'type':0}),
                 'N defect' : EditPointTags(defect_tags, data_fields={'type':1}),
                }

summary = Summary(write_file='summary.json')

In [54]:
atom_tags.point_tags.color_scheme = 'category'
atom_tags.point_tags.color_scale.min = 0
atom_tags.point_tags.color_scale.max = 10
toolbox.button_width = '87px'

In [55]:
def on_new_image(*args):
    has_defect_tags.t = np.where(n_defects)[0]
    skipped_tags.t = np.where(area_tags.area < .9)[0]
    
    if len(has_defect_tags.t) > 0:
        timeline.frame_index = int(min(has_defect_tags.t))
    else:
        timeline.frame_index = 0

In [56]:
nanotag_data.read_data()
on_new_image()
canvas.reset()

tools_column = widgets.VBox([image_file_collection, nanotag_data, gaussian_filter, atom_tags, toolbox])
canvas_box = widgets.VBox([canvas, timeline])
summary_column = widgets.VBox([summary])

widgets.HBox([canvas_box, tools_column, summary_column])

In [30]:
from nanotag.events import KeyEvents

events = KeyEvents(canvas_box)

def toggle_point_visible(*args):
    atom_tags.point_tags.visible = not atom_tags.point_tags.visible

def toggle_defects_visible(*args):
    defect_tags.point_tags.visible = not defect_tags.point_tags.visible
    
events.callbacks = {'ArrowLeft' : timeline.previous_frame,
                    'ArrowRight' : timeline.next_frame,
                    'ArrowUp' : image_file_collection.next_file,
                    'ArrowDown' : image_file_collection.previous_file,
                    'r' : canvas.reset,
                    'z' : lambda : toolbox.toggle_tool('BoxZoom'),
                    'p' : lambda : toolbox.toggle_tool('Pan'),
                    'l' : toggle_point_visible,
                    'd' : toggle_defects_visible,
                   }